# Format Data

This notebook splits the cleaned data (ie the three "clean.txt" files) into chunks of 10 words. It then uses LatinBERT to extract the sentence representation of each chunk and saves the features along with its classification in a dataframe that gets outputted to a csv file. The csv is formatted so the model can take data as input.

In [1]:
import pandas as pd

In [2]:
# From gen_berts.py
import argparse, sys
from cltk.tokenizers.lat.lat import LatinWordTokenizer as WordTokenizer
from cltk.tokenizers.lat.lat import LatinPunktSentenceTokenizer as SentenceTokenizer
from tensor2tensor.data_generators import text_encoder
import numpy as np
import torch
from torch import nn
from transformers import BertModel, BertPreTrainedModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LatinBERT():

	def __init__(self, tokenizerPath=None, bertPath=None):
		encoder = text_encoder.SubwordTextEncoder(tokenizerPath)
		self.wp_tokenizer = LatinTokenizer(encoder)
		self.model = BertLatin(bertPath=bertPath)
		self.model.to(device)

	def get_batches(self, sentences, max_batch, tokenizer):

			maxLen=0
			for sentence in sentences:
				length=0
				for word in sentence:
					toks=tokenizer.tokenize(word)
					length+=len(toks)

				if length> maxLen:
					maxLen=length

			all_data=[]
			all_masks=[]
			all_labels=[]
			all_transforms=[]

			for sentence in sentences:
				tok_ids=[]
				input_mask=[]
				labels=[]
				transform=[]

				all_toks=[]
				n=0
				for idx, word in enumerate(sentence):
					toks=tokenizer.tokenize(word)
					all_toks.append(toks)
					n+=len(toks)

				cur=0
				for idx, word in enumerate(sentence):
					toks=all_toks[idx]
					ind=list(np.zeros(n))
					for j in range(cur,cur+len(toks)):
						ind[j]=1./len(toks)
					cur+=len(toks)
					transform.append(ind)

					tok_ids.extend(tokenizer.convert_tokens_to_ids(toks))

					input_mask.extend(np.ones(len(toks)))
					labels.append(1)

				all_data.append(tok_ids)
				all_masks.append(input_mask)
				all_labels.append(labels)
				all_transforms.append(transform)

			lengths = np.array([len(l) for l in all_data])

			# Note sequence must be ordered from shortest to longest so current_batch will work
			ordering = np.argsort(lengths)
			
			ordered_data = [None for i in range(len(all_data))]
			ordered_masks = [None for i in range(len(all_data))]
			ordered_labels = [None for i in range(len(all_data))]
			ordered_transforms = [None for i in range(len(all_data))]
			

			for i, ind in enumerate(ordering):
				ordered_data[i] = all_data[ind]
				ordered_masks[i] = all_masks[ind]
				ordered_labels[i] = all_labels[ind]
				ordered_transforms[i] = all_transforms[ind]

			batched_data=[]
			batched_mask=[]
			batched_labels=[]
			batched_transforms=[]

			i=0
			current_batch=max_batch

			while i < len(ordered_data):

				batch_data=ordered_data[i:i+current_batch]
				batch_mask=ordered_masks[i:i+current_batch]
				batch_labels=ordered_labels[i:i+current_batch]
				batch_transforms=ordered_transforms[i:i+current_batch]

				max_len = max([len(sent) for sent in batch_data])
				max_label = max([len(label) for label in batch_labels])

				for j in range(len(batch_data)):
					
					blen=len(batch_data[j])
					blab=len(batch_labels[j])

					for k in range(blen, max_len):
						batch_data[j].append(0)
						batch_mask[j].append(0)
						for z in range(len(batch_transforms[j])):
							batch_transforms[j][z].append(0)

					for k in range(blab, max_label):
						batch_labels[j].append(-100)

					for k in range(len(batch_transforms[j]), max_label):
						batch_transforms[j].append(np.zeros(max_len))

				batched_data.append(torch.LongTensor(batch_data))
				batched_mask.append(torch.FloatTensor(batch_mask))
				batched_labels.append(torch.LongTensor(batch_labels))
				batched_transforms.append(torch.FloatTensor(batch_transforms))

				bsize=torch.FloatTensor(batch_transforms).shape
				
				i+=current_batch

				# adjust batch size; sentences are ordered from shortest to longest so decrease as they get longer
				if max_len > 100:
					current_batch=12
				if max_len > 200:
					current_batch=6

			return batched_data, batched_mask, batched_transforms, ordering


	def get_berts(self, raw_sents):
		sents=convert_to_toks(raw_sents)
		batch_size=32
		batched_data, batched_mask, batched_transforms, ordering=self.get_batches(sents, batch_size, self.wp_tokenizer)

		ordered_preds=[]
		for b in range(len(batched_data)):
			size=batched_transforms[b].shape
			b_size=size[0]
			berts=self.model.forward(batched_data[b], attention_mask=batched_mask[b], transforms=batched_transforms[b])
			berts=berts.detach()
			berts=berts.cpu()
			for row in range(b_size):
				ordered_preds.append([np.array(r) for r in berts[row]])

		preds_in_order = [None for i in range(len(sents))]


		for i, ind in enumerate(ordering):
			preds_in_order[ind] = ordered_preds[i]


		bert_sents=[]

		for idx, sentence in enumerate(sents):
			bert_sent=[]

			bert_sent.append(("[CLS]", preds_in_order[idx][0] ))

			for t_idx in range(1, len(sentence)-1):
				token=sentence[t_idx]
				
				pred=preds_in_order[idx][t_idx]
				bert_sent.append((token, pred ))

			bert_sent.append(("[SEP]", preds_in_order[idx][len(sentence)-1] ))

			bert_sents.append(bert_sent)

		return bert_sents




class LatinTokenizer():
	def __init__(self, encoder):
		self.vocab={}
		self.reverseVocab={}
		self.encoder=encoder

		self.vocab["[PAD]"]=0
		self.vocab["[UNK]"]=1
		self.vocab["[CLS]"]=2
		self.vocab["[SEP]"]=3
		self.vocab["[MASK]"]=4

		for key in self.encoder._subtoken_string_to_id:
			self.vocab[key]=self.encoder._subtoken_string_to_id[key]+5
			self.reverseVocab[self.encoder._subtoken_string_to_id[key]+5]=key


	def convert_tokens_to_ids(self, tokens):
		wp_tokens=[]
		for token in tokens:
			if token == "[PAD]":
				wp_tokens.append(0)
			elif token == "[UNK]":
				wp_tokens.append(1)
			elif token == "[CLS]":
				wp_tokens.append(2)
			elif token == "[SEP]":
				wp_tokens.append(3)
			elif token == "[MASK]":
				wp_tokens.append(4)

			else:
				wp_tokens.append(self.vocab[token])

		return wp_tokens

	def tokenize(self, text):
		tokens=text.split(" ")
		wp_tokens=[]
		for token in tokens:

			if token in {"[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"}:
				wp_tokens.append(token)
			else:

				wp_toks=self.encoder.encode(token)

				for wp in wp_toks:
					wp_tokens.append(self.reverseVocab[wp+5])

		return wp_tokens

def convert_to_toks(sents):

	sent_tokenizer = SentenceTokenizer()
	word_tokenizer = WordTokenizer()

	all_sents=[]

	for data in sents:
		text=data.lower()

		sents=sent_tokenizer.tokenize(text)
		for sent in sents:
			tokens=word_tokenizer.tokenize(sent)
			filt_toks=[]
			filt_toks.append("[CLS]")
			for tok in tokens:
				if tok != "":
					filt_toks.append(tok)
			filt_toks.append("[SEP]")

			all_sents.append(filt_toks)

	return all_sents




class BertLatin(nn.Module):

	def __init__(self, bertPath=None):
		super(BertLatin, self).__init__()

		self.bert = BertModel.from_pretrained(bertPath)
		self.bert.eval()
		
	def forward(self, input_ids, token_type_ids=None, attention_mask=None, transforms=None):

		input_ids = input_ids.to(device)
		attention_mask = attention_mask.to(device)
		transforms = transforms.to(device)
		sequence_outputs, pooled_outputs = self.bert.forward(input_ids, token_type_ids=None, attention_mask=attention_mask)

		all_layers=sequence_outputs
		out=torch.matmul(transforms,all_layers)
		return out

In [3]:
# creates dataframe of latin sentences, if it is history or myth, and its features extracted from LatinBERT
# sents = list of sentences, isHistory = 1/0
def getDF(sents, isHistory):
    bertPath="latin-bert-master/models/latin_bert/"
    tokenizerPath="latin-bert-master/models/subword_tokenizer_latin/latin.subword.encoder"

    bert=LatinBERT(tokenizerPath=tokenizerPath, bertPath=bertPath)
    
    bert_sents=bert.get_berts(sents)

    rows = set([])

    count = 0
    for sent in bert_sents:
        vector = np.array([])
        for (token, bert) in sent[1:-1]:
            vector = np.concatenate((vector, bert), axis=0)
        temp = [str(sents[count]), int(isHistory)]+[float(i) for i in vector[:7680]] # tokenizer sometimes splits words, truncate as solution
        rows.add(tuple(temp))
        count += 1
    return pd.DataFrame(rows, columns = ['latin', 'isHistory']+[i for i in range(7680)])

In [43]:
# load history data
wordList = []
file = 'data/history/clean.txt'

with open(file, "r") as f:
    wordList = f.read().split()

sentences = set([])

for i in range(0, len(wordList), 10):
    sentences.add(' '.join(wordList[i:i+10]))

# sentences is a set of strings where each string is 10 words of latin
len(sentences)

69235

In [ ]:
# output features of history data as a series of dataframes --> csv files
listSentences = list(sentences)

for i in range(0, 60000, 3000):
    getDF(listSentences[i:i+1000], 1).to_csv('history-dfs/'+str(i//1000)+'.csv')
    print(i)

26000
27000


In [4]:
# load myth data
wordList = []
file = 'data/myth/clean.txt'

with open(file, "r") as f:
    wordList = f.read().split()

sentencesM = set([])

for i in range(0, len(wordList), 10):
    sentencesM.add(' '.join(wordList[i:i+10]))

# sentences is a set of strings where each string is 10 words of latin
len(sentencesM)

30464

In [5]:
# output features of myth data as a series of dataframes --> csv files
listSentencesM = list(sentencesM)

#only get 10,000 data points
for i in range(0, 21000, 2000):
    getDF(listSentencesM[i:i+1000], 0).to_csv('myth-dfs/'+str(i//1000)+'.csv')
    print(i)

/var/folders/rv/70f4qh790dx4d3n6r7_9bhs80000gn/T/ipykernel_40639/3253452958.py:125: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  batched_transforms.append(torch.FloatTensor(batch_transforms))


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000


In [10]:
# load nero data
wordList = []
file = 'data/nero/clean.txt'

with open(file, "r") as f:
    wordList = f.read().split()

sentences = set([])

for i in range(0, len(wordList), 10):
    sentences.add(' '.join(wordList[i:i+10]))

# sentences is a set of strings where each string is 10 words of latin
len(sentences)

2338

In [12]:
# output features of nero data as a series of dataframes --> csv files
listSentencesN = list(sentences)

for i in range(0, len(sentences), 1000):
    getDF(listSentencesN[i:i+1000], 0).to_csv('nero-dfs/'+str(i//1000)+'.csv')
    print(i)

0
1000
2000


In [4]:
# load nero - annals data
wordList = []
file = 'data/nero/clean-annals.txt'

with open(file, "r") as f:
    wordList = f.read().split()

sentences = set([])

for i in range(0, len(wordList), 10):
    sentences.add(' '.join(wordList[i:i+10]))

# sentences is a set of strings where each string is 10 words of latin
len(sentences)

1573

In [5]:
# output features of nero-annals data as a dataframe --> csv
listSentencesN = list(sentences)

getDF(listSentencesN, 0).to_csv('nero-dfs/annals.csv')

/var/folders/rv/70f4qh790dx4d3n6r7_9bhs80000gn/T/ipykernel_43877/3253452958.py:125: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  batched_transforms.append(torch.FloatTensor(batch_transforms))


In [6]:
# load nero - bio data
wordList = []
file = 'data/nero/clean-biography.txt'

with open(file, "r") as f:
    wordList = f.read().split()

sentences = set([])

for i in range(0, len(wordList), 10):
    sentences.add(' '.join(wordList[i:i+10]))

# sentences is a set of strings where each string is 10 words of latin
len(sentences)

766

In [7]:
# output features of nero-annals data as a dataframe --> csv
listSentencesN = list(sentences)

getDF(listSentencesN, 0).to_csv('nero-dfs/biography.csv')